<a href="https://colab.research.google.com/github/ppaulo17/Mine/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [7]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-11-24 22:55:33--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  94.7MB/s    in 0.3s    

2025-11-24 22:55:33 (94.7 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [9]:
# code
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index


book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index

df_ratings_filtered = df_ratings[
    (df_ratings['user'].isin(active_users)) &
    (df_ratings['isbn'].isin(popular_books))
]

df_merged = pd.merge(df_ratings_filtered, df_books, on='isbn') #fusiona los datos

df_merged = df_merged.drop_duplicates(['title', 'user']) #elimina duplicados

book_user_mat = df_merged.pivot(index='title', columns='user', values='rating').fillna(0) #crea matriz

book_user_mat_sparse = csr_matrix(book_user_mat.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_user_mat_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
def get_recommends(book = ""):
  if book not in book_user_mat.index:
    return "Book not found in database"

  book_loc = book_user_mat.index.get_loc(book)
  book_vector = book_user_mat.iloc[book_loc].values.reshape(1, -1)

  distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

  recommended_books = []

  # This loop starts from 1 to exclude the book itself
  for i in range(1, len(distances.flatten())):
    title = book_user_mat.index[indices.flatten()[i]]
    distance = distances.flatten()[i]
    recommended_books.append((title, distance))

  # Sort by distance in descending order (as per the example output and test expectations)
  recommended_books = sorted(recommended_books, key=lambda x: x[1], reverse=True)

  # Return the input book title and the list of recommendations, as expected by the test function
  return [book, recommended_books]

In [11]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [("I'll Be Seeing You", np.float32(0.8016211)), ('The Weight of Water', np.float32(0.77085835)), ('The Surgeon', np.float32(0.7699411)), ('I Know This Much Is True', np.float32(0.7677075)), ('The Lovely Bones: A Novel', np.float32(0.7234864))]]
You passed the challenge! 🎉🎉🎉🎉🎉
